# This notebook gets files from the AWS bucket and repopulates the
# ds_cases table using the boto3 library by getting the list of filenames
# from the s3 bucket and sending GET requests to the endpoint (pdf-ocr)

In [1]:
# importing a library that simplifes interactions with AWS

!pip install boto3 --quiet

In [2]:
import requests as re
import pandas as pd
from boto3.session import Session

df = pd.read_csv('manually_scrapped.csv')

In [3]:
# creating a session and connecting to our s3 bucket. you have to manually enter the 
# access keys - or at least i couldn't get them from .env

ACCESS_KEY = 'your-access-key'
SECRET_ACCESS_KEY = 'your-secret-access-key'

session = Session(aws_access_key_id=ACCESS_KEY, 
                  aws_secret_access_key=SECRET_ACCESS_KEY)

s3 = session.resource('s3')
your_bucket = s3.Bucket('hrf-asylum-cases')

In [4]:
# getting the filenames of the pdfs in the s3 hrf-asylum-cases bucket
# these can be used in the make_fields function along with the files to
# repopulate the ds_cases table.

filenames = []

for s3_file in your_bucket.objects.all():
    filenames.append(s3_file.key)

In [6]:
# reloading the cases into the ds_cases table using the endpoint setup
# through elasticbeanstalk


API_URL = 'http://labs37-hrf-asylum-b-dev.us-east-1.elasticbeanstalk.com/pdf-ocr'        # your endpoint will go here

for file in filenames:
    file = file.strip('.pdf')
    re.get(f'{API_URL}/{file}')

KeyboardInterrupt: 

In [9]:
# getting the files stored in the bucket. note they currently download to
# the notebooks folder. change in open(). these files are needed
# to check and calibrate the accuracy of the ocr

for i in range(len(df)):
    r = re.get(df['AWS link'][i])
    print(r)
    open('../' + name, 'wb').write(r.content)